Primeiramente, é necessária a leitura dos 3 arquivos, inserindo as informações em um vetor:

In [1]:
import codecs
with codecs.open("imdb_labelled.txt", "r",  "utf-8") as arquivo:
    vetor = []
    for linha in arquivo:
        vetor.append(linha)
with codecs.open("amazon_cells_labelled.txt", "r",  "utf-8") as arquivo:
    for linha in arquivo:
        vetor.append(linha)
with codecs.open("yelp_labelled.txt", "r",  "utf-8") as arquivo:
    for linha in arquivo:
        vetor.append(linha)

Depois, devemos retirar cada quebra de linha no final de cada linha, ou seja, os '\n'.

In [2]:
vetor = [ x[:-1] for x in vetor ]

vetor = ([s.replace('&', '').replace(' - ', '').replace('.', '').replace(',', '').replace('!', '').
          replace('+', '')for s in vetor])

A seguir, retiramos os dois últimos caracteres sobrando apenas o nosso comentário. Depois, passamos ele para lowercase.

In [11]:
import nltk
TextosQuebrados = [ x[:-4] for x in vetor ]

TextosQuebrados = map(lambda X:X.lower(),TextosQuebrados)

#TextosQuebrados = [x.split(' ') for x in TextosQuebrados]

TextosQuebrados = [nltk.tokenize.word_tokenize(frase) for frase in TextosQuebrados]


In [12]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')

stemmer = nltk.stem.RSLPStemmer()

dicionario = set()

for comentarios in TextosQuebrados:
    validas = [stemmer.stem(palavra) for palavra in comentarios if palavra not in stopwords and len(palavra) > 0]
    dicionario.update(validas)

    
totalDePalavras = len(dicionario)
tuplas = zip(dicionario, xrange(totalDePalavras))
tradutor = {palavra:indice for palavra,indice in tuplas}
        
def vetorizar_texto(texto, tradutor, stemmer):
    vetor = [0] * len(tradutor)
    for palavra in texto:
        if len(palavra) > 0:
            raiz = stemmer.stem(palavra)
            if raiz in tradutor:
                posicao = tradutor[raiz]
                vetor[posicao] += 1

    return vetor

vetoresDeTexto = [vetorizar_texto(texto, tradutor,stemmer) for texto in TextosQuebrados]
X = vetoresDeTexto

In [13]:
Y = [ x[-1:] for x in vetor ]

In [14]:
porcentagem_de_treino = 0.8

tamanho_do_treino = porcentagem_de_treino * len(Y)
tamanho_de_validacao = len(Y) - tamanho_do_treino

treino_dados = X[0:int(tamanho_do_treino)]
treino_marcacoes = Y[0:int(tamanho_do_treino)]

validacao_dados = X[int(tamanho_do_treino):]
validacao_marcacoes = Y[int(tamanho_do_treino):]

fim_de_teste = tamanho_do_treino + tamanho_de_validacao
teste_dados = X[int(tamanho_do_treino):int(fim_de_teste)]
teste_marcacoes = Y[int(tamanho_do_treino):int(fim_de_teste)]

Foi decidida a abordagem por poly SCV

In [15]:
""" from sklearn import svm
from sklearn.model_selection import cross_val_score
k = 10

# Implement poly SVC 
poly_svc = svm.SVC(kernel='linear')
accuracy_poly_svc = cross_val_score(poly_svc, treino_dados, treino_marcacoes, cv=k, scoring='accuracy')
print('poly_svc: ', accuracy_poly_svc.mean()) """

" from sklearn import svm\nfrom sklearn.model_selection import cross_val_score\nk = 10\n\n# Implement poly SVC \npoly_svc = svm.SVC(kernel='linear')\naccuracy_poly_svc = cross_val_score(poly_svc, treino_dados, treino_marcacoes, cv=k, scoring='accuracy')\nprint('poly_svc: ', accuracy_poly_svc.mean()) "

Resultado - Poly: 

Os 3: Após 10 minutos rodando, foi decidido parar o teste

IMdB: 0.51750234411626805

Amazon: 0.51125019534302241

Yelp: 0.56500429754649173

Resultado - Linear:

Os 3: 0.7745982496802607 (5 minutos)

IMdB: 0.72168288013752147

Amazon: 0.78869745272698855

Yelp: 0.77492342553523996

In [24]:
def fit_and_predict(modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes):
	modelo.fit(treino_dados, treino_marcacoes)

	resultado = modelo.predict(teste_dados)
	acertos = (resultado == teste_marcacoes)

	total_de_acertos = sum(acertos)
	total_de_elementos = len(teste_dados)
	taxa_de_acerto = float(total_de_acertos) / float(total_de_elementos)

	print(taxa_de_acerto)
	return taxa_de_acerto


resultados = {}

from sklearn.naive_bayes import MultinomialNB
classificador = MultinomialNB()

resultadoMultinomial = fit_and_predict(classificador, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

0.805324459235


Com maior refinamento de dados:
    
MultinomialNB:
    
Todos: 0.808652246256

In [26]:
from sklearn.ensemble import AdaBoostClassifier
classificador = AdaBoostClassifier(n_estimators=100)

resultado = fit_and_predict(classificador, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

0.527454242928


Adaboost:

0.527454242928

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

classificador = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(treino_dados, treino_marcacoes)
resultado = fit_and_predict(classificador, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)


0.737104825291


GradientBoostingClassifier:

Todos: 0.77870216306156403

In [28]:
from sklearn.naive_bayes import GaussianNB


classificador = GaussianNB()
resultado = fit_and_predict(classificador, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)


0.665557404326


Gaussiano:

0.665557404326

In [29]:
from sklearn.naive_bayes import BernoulliNB
classificador = BernoulliNB()
resultado = fit_and_predict(classificador, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

0.801996672213


Bernoulli: 0.801996672213